In [39]:
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import statistics
import datetime, time
import yfinance as yf
import pandas as pd
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
import random

In [40]:
def sign(x):
    return 1 if x>0 else -1
def BSM_call(S0, strike, y_frac, sigma, rate):
    nd = statistics.NormalDist(0,1)
    d1 = (np.log(S0/strike)+(rate+sigma*sigma/2)*y_frac)/sigma/math.sqrt(y_frac)
    d2 = d1 - sigma * math.sqrt(y_frac)
    return S0*nd.cdf(d1) -  strike*math.exp((-1)*rate*y_frac)*nd.cdf(d2)

def BSM_put(S0, strike, y_frac, sigma, rate):
    nd = statistics.NormalDist(0,1)
    d1 = (np.log(S0/strike)+(rate+sigma*sigma/2)*y_frac)/sigma/math.sqrt(y_frac)
    d2 = d1 - sigma * math.sqrt(y_frac)
    return -S0*nd.cdf(-d1) +  strike*math.exp((-1)*rate*y_frac)*nd.cdf(-d2)
def hurst(ts):
    npts = np.array(ts)
    logs = log(npts[:-1]/npts[1:])
    groups = np.arange(int(len(logs) / 10), 1, -1)
    RS = np.zeros(len(groups)+1)
    line = np.zeros(len(groups)+1)
    for group in groups:
        sub_RS = np.zeros(group)
        stdev = np.zeros(group)
        line[group-1] = len(logs)/group
        for sub in range(group):
            RSmax, RSmin = 0, 0
            sub_group = np.array(logs[int(len(logs)/group)*sub : int(len(logs)/group*(sub+1))])
            for in_sub in range(1, int(len(logs)/group)):
                sub_RS[sub] += sub_group[in_sub]
                RSmax = max(sub_RS[sub], RSmax)
                RSmin = min(sub_RS[sub], RSmin)
            sub_RS[sub] = RSmax - RSmin
            stdev[sub] = std(sub_group)
        a = sub_RS / stdev
        RS[group-1] = a.mean()
    poly = polyfit(log(line[1:]), log(RS[1:]), 1)
    H = poly[0] 
    return H

In [61]:
def getparams(ticker, strt, s1, endt):
    format = '%Y-%m-%d'
    dtstrt = datetime.datetime.strptime(strt, format)
    dtendt = datetime.datetime.strptime(endt, format)
    time_delta = dtendt-dtstrt
    time_marshall_as_year = time_delta.total_seconds() / 365.25 / 86400
    tau = time_marshall_as_year / len(ticker)
    
    ticks = np.arange(1,len(ticker)+1)
    obs = len(ticker)
    log_data = np.zeros(len(ticker))
    div = ticker[0]
    log_data = np.log(ticker / div)
    XT = log_data[len(log_data)-1]
    mean = XT / len(log_data)
    var = 0
    for i in range(1, len(log_data)):
        var += (log_data[i] - log_data[i-1]) * (log_data[i] - log_data[i-1])
    var /= len(log_data) * tau
    mu = XT/time_marshall_as_year
    sigma = math.sqrt(var)
    hrst = hurst(ticker)
    
    call = "Call" if mu > 0 else "Put"
    r = 0.025
    s0 = ticker[0]
    s01 = ticker[len(ticks)-1]
    tmy = time_marshall_as_year
    bsm = BSM_call(s01, s01, tmy, sigma, r) if mu > 0 else BSM_put(s01, s01, tmy, sigma, r)
    res = (sign(mu)*(-s01 + s1) - bsm) / bsm
    res = res if res > -1 else -1
    sgn = "+" if mu>0 else "-"
        
    cbsm = BSM_call(s01, s01, tmy, sigma, r)
    cr = (s1 - s01 - cbsm) / cbsm
    cr = cr if cr > -1 else -1
        
    pbsm = BSM_put(s01, s01, tmy, sigma, r)
    pr = (s01 - s1 - pbsm) / pbsm
    pr = pr if pr > -1 else -1
        
    rsgn = sign(random.uniform(-3, 3))
    rcall = "Call" if rsgn > 0 else "Put"
    rbsm = BSM_call(s01, s01, tmy, sigma, r) if rsgn > 0 else BSM_put(s01, s01, tmy, sigma, r)
    rr = (rsgn*(-s01 + s1) - rbsm) / rbsm
    rr = rr if rr > -1 else -1
        
    print ("s0 = ", s0,"s01 = ", s01, "s1 = ", s1, 'bsm = ', bsm, 'res = ', res)
    #print('Hurst =',hrst,'Sigma =',sigma,'Mu =',mu,'BSM_res = ',res)
    print()
    return obs, mu, sigma, hrst, call, res, cr, pr, rcall, rr

In [42]:
time_list = []
for y in range(19, 22):
    for m in range (1, 13):
        time_list.append("20"+str(y)+"-"+str(m))
time_list=time_list[5:-7] #len 24

In [244]:
results = []
litera = 'Z' #X,Y,Z
index = 0
ticker_list = pd.read_csv('C:/Users/prof-/OneDrive/Рабочий стол/ПНИПУ/FinanceMath/tickers/'+litera+'_ticks.csv', index_col=0)
a = len(ticker_list['Символ'])
b = len(time_list[:-7])
c = a * b
print(a)
print(b)
print(c)

15
17
255


In [245]:
print(index)

0


In [246]:
for i in range(index+1, len(ticker_list['Символ'])-1):
    
    try:
        tick = ticker_list['Символ'][i]
        index = i
        data = yf.download(tick, start=time_list[0]+"-16", end=time_list[23]+"-16", interval="1h")
        zero = 1 / len(data)
        
        for j in range(18):
            strt = time_list[j]+"-15"
            endt = time_list[j+3]+"-15"
            strt1 = time_list[j+5]+"-16"
            endt1 = time_list[j+6]+"-16"
            dt = data['Adj Close'].loc[strt1:endt1]
            s1 = dt[len(dt)-1]
            
            dt = data['Adj Close'].loc[strt:endt]
            results.append(getparams(dt, strt, s1, endt1))
            
    except (ZeroDivisionError, TypeError, IndexError, NameError, KeyError):
        print("Error")

[*********************100%***********************]  1 of 1 completed
s0 =  88.12000274658203 s01 =  73.4800033569336 s1 =  76.80999755859375 bsm =  5.021131052448808 res =  -1

s0 =  93.8949966430664 s01 =  69.1500015258789 s1 =  84.69000244140625 bsm =  5.4120756548333695 res =  -1

s0 =  74.86000061035156 s01 =  75.43000030517578 s1 =  90.48999786376953 bsm =  6.10180830483889 res =  1.4681204007426432

s0 =  74.0199966430664 s01 =  75.76000213623047 s1 =  54.7400016784668 bsm =  5.803615839619248 res =  -1

s0 =  69.4800033569336 s01 =  84.0 s1 =  68.91999816894531 bsm =  5.678952811442876 res =  -1

s0 =  74.44000244140625 s01 =  90.48999786376953 s1 =  74.58000183105469 bsm =  5.446326464889751 res =  -1

s0 =  76.90499877929688 s01 =  64.23999786376953 s1 =  82.51000213623047 bsm =  5.428705670319829 res =  -1

s0 =  85.05000305175781 s01 =  68.83000183105469 s1 =  87.36000061035156 bsm =  8.935175180374507 res =  -1

s0 =  90.45999908447266 s01 =  74.58000183105469 s1 =  87.8799

In [248]:
dtfrm = pd.DataFrame(results[1:],columns=['Observations', 'Trend', 'Vol', 'Hurst', 'Option', 'Res', 'CallOnly', 'PutOnly', 'RandDec', 'rRes'])
print(index)
dtfrm = dtfrm[dtfrm['Observations'] >250 ]
dtfrm = dtfrm[dtfrm['Vol'] < 1]
dtfrm.describe()

13


,Observations,Trend,Vol,Hurst,Res,CallOnly,PutOnly,rRes
count,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000
mean,445.075472,0.127801,0.282974,0.568233,-0.036731,0.470574,0.168690,0.583636
std,11.743818,0.441459,0.115700,0.048689,1.784622,1.492921,2.708177,2.069143
min,421.000000,-1.174983,0.122051,0.480663,-1.000000,-1.000000,-1.000000,-1.000000
25%,441.000000,-0.042329,0.179657,0.528536,-1.000000,-1.000000,-1.000000,-1.000000
50%,447.000000,0.241060,0.271287,0.566792,-1.000000,0.214997,-1.000000,0.017458
75%,452.000000,0.427402,0.366869,0.601097,0.500153,1.468120,-0.302392,1.473667
max,462.000000,0.962843,0.526606,0.677685,9.791896,5.954101,10.502140,9.791896


In [249]:
writer = pd.ExcelWriter('C:/Users/prof-/OneDrive/Рабочий стол/ПНИПУ/FinanceMath/tickers/'+litera+'_specs.xlsx', engine='xlsxwriter')
dtfrm.to_excel(writer, 'Sheet1')
writer.save()